# #IPL 2019 winner prediction using ipl data from 2008 - 2018.


# The folloing are the imports

### fastai.structured
For this prediction i have used fastai library which handles dates and encodes catagorical veriables with ease.  

### pandas 
for dealing with dataframes 

### sklearn 
for importing ML algorithms like RandomForest

In [25]:
from fastai.structured  import *
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import warnings

#### Reading the CSV files and storing them in data(training) and test(testing).

In [26]:
train=pd.read_csv('/home/aravind/Documents/fastai/courses/matches.csv', index_col=0)
test=pd.read_csv('/home/aravind/Documents/fastai/courses/test.csv', index_col=0)
train.tail()

,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
id,,,,,,,,,,,,,,,,,
11347,2019,Mumbai,05/05/19,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,9,HH Pandya,Wankhede Stadium,Nanda Kishore,O Nandan,S Ravi
11412,2019,Chennai,07/05/19,Chennai Super Kings,Mumbai Indians,Chennai Super Kings,bat,normal,0,Mumbai Indians,0,6,AS Yadav,M. A. Chidambaram Stadium,Nigel Llong,Nitin Menon,Ian Gould
11413,2019,Visakhapatnam,08/05/19,Sunrisers Hyderabad,Delhi Capitals,Delhi Capitals,field,normal,0,Delhi Capitals,0,2,RR Pant,ACA-VDCA Stadium,NaN,NaN,NaN
11414,2019,Visakhapatnam,10/05/19,Delhi Capitals,Chennai Super Kings,Chennai Super Kings,field,normal,0,Chennai Super Kings,0,6,F du Plessis,ACA-VDCA Stadium,Sundaram Ravi,Bruce Oxenford,Chettithody Shamshuddin
11415,2019,Hyderabad,12/05/19,Mumbai Indians,Chennai Super Kings,Mumbai Indians,bat,normal,0,Mumbai Indians,1,0,JJ Bumrah,Rajiv Gandhi Intl. Cricket Stadium,Nitin Menon,Ian Gould,Nigel Llong


#### As we have seen from the above table we dont reqire the columns  'win_by_runs','win_by_wickets' and 'player_of_match' since our aim is to predict the future outcome.
#### Since columns 'umpire1' , 'umpire2' and 'umpire3' have nothing to do with the target variable('winner'), we can drop them as well.  
### Here our target variable is 'winner'

#### We have stored our train data in a variable called as 'data1' so that we could access our unmuated dataset at any point of time in the future.

In [28]:
data1=train.drop(['win_by_runs','win_by_wickets','player_of_match','umpire3','umpire2','umpire1'], axis = 1)
test=test.drop(['win_by_runs','win_by_wickets','player_of_match','umpire3','umpire2','umpire1'], axis = 1)
data1

,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,venue
id,,,,,,,,,,,
1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal"
2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,Maharashtra Cricket Association Stadium
3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,Saurashtra Cricket Association Stadium
4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,Holkar Cricket Stadium
5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,M Chinnaswamy Stadium
...,...,...,...,...,...,...,...,...,...,...,...
11347,2019,Mumbai,05/05/19,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,Wankhede Stadium
11412,2019,Chennai,07/05/19,Chennai Super Kings,Mumbai Indians,Chennai Super Kings,bat,normal,0,Mumbai Indians,M. A. Chidambaram Stadium
11413,2019,Visakhapatnam,08/05/19,Sunrisers Hyderabad,Delhi Capitals,Delhi Capitals,field,normal,0,Delhi Capitals,ACA-VDCA Stadium


#### Cheacking weather our dataset has any NULL values.

In [4]:
data1.isnull().sum()

season           0
city             7
date             0
team1            0
team2            0
toss_winner      0
toss_decision    0
result           0
dl_applied       0
winner           4
venue            0
dtype: int64

#### The columns 'city' and 'winner' have multipel NULL vales so we are going to fill it with the most recuring value of the column since its a catogorical variable.
#### In the case of contineous numeric variables we can use Mean of the variable instead of the NULL vales as it doesnt effect the performance of the model. 

In [5]:
data1['city'].fillna(train['city'].value_counts().idxmax(), inplace=True)
data1['winner'].fillna(train['winner'].value_counts().idxmax(), inplace=True)

#### While dealing with dates our fastai library has a function which converts the date into many other columns like 'Year',  'Month',  'Week',  'Day',  'Dayofweek',  'Dayofyear',  'Is_month_end',  'Is_month_start', 'Is_quarter_end',  'Is_quarter_start',  'Is_year_end',  'Is_year_start'  and 'Elapsed'  which help the Machine learning model to take all the features of date into training. 
 
##### For more information on the 'add_datepart'  read fastai docs.
#### github.com/fastai/fastai

In [6]:
add_datepart(data1, 'date')
add_datepart(test, 'date')

In [7]:
data1.head()

,season,city,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,venue,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
id,,,,,,,,,,,,,,,,,,,,,
1,2017,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal",...,5,2,95,False,False,False,False,False,False,1491350400
2,2017,Pune,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,Maharashtra Cricket Association Stadium,...,6,3,96,False,False,False,False,False,False,1491436800
3,2017,Rajkot,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,Saurashtra Cricket Association Stadium,...,7,4,97,False,False,False,False,False,False,1491523200
4,2017,Indore,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,Holkar Cricket Stadium,...,8,5,98,False,False,False,False,False,False,1491609600
5,2017,Bangalore,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,M Chinnaswamy Stadium,...,8,5,98,False,False,False,False,False,False,1491609600


#### Since our aim is to predict the outcome of 2019 IPL we need to neglect the 2019 ipl data from our dataset.

In [8]:
data1 = data1[data1.season != 2019]

#### Just making sure that the 2019 data doesnt exit in the 'data1' dataframe    

In [9]:
data1.tail()

,season,city,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,venue,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
id,,,,,,,,,,,,,,,,,,,,,
7949,2018,Pune,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,field,normal,0,Chennai Super Kings,Maharashtra Cricket Association Stadium,...,20,6,140,False,False,False,False,False,False,1526774400
7950,2018,Mumbai,Sunrisers Hyderabad,Chennai Super Kings,Chennai Super Kings,field,normal,0,Chennai Super Kings,Wankhede Stadium,...,22,1,142,False,False,False,False,False,False,1526947200
7951,2018,Kolkata,Kolkata Knight Riders,Rajasthan Royals,Rajasthan Royals,field,normal,0,Kolkata Knight Riders,Eden Gardens,...,23,2,143,False,False,False,False,False,False,1527033600
7952,2018,Kolkata,Sunrisers Hyderabad,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Sunrisers Hyderabad,Eden Gardens,...,25,4,145,False,False,False,False,False,False,1527206400
7953,2018,Mumbai,Sunrisers Hyderabad,Chennai Super Kings,Chennai Super Kings,field,normal,0,Chennai Super Kings,Wankhede Stadium,...,27,6,147,False,False,False,False,False,False,1527379200


#### 'train_cats' is one of the best features of the fastai library  
#### it change any columns of strings in a panda's dataframe to a column of categorical values. This applies the changes inplace.
#### 'apply_cats' make sure that the encoding used for training set is same as that of test set. 

In [27]:
train_cats(data1)
train_cats(test)
apply_cats(test,data1)
data1

,season,city,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,venue,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
id,,,,,,,,,,,,,,,,,,,,,
1,2017,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal",...,5,2,95,False,False,False,False,False,False,1491350400
2,2017,Pune,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,Maharashtra Cricket Association Stadium,...,6,3,96,False,False,False,False,False,False,1491436800
3,2017,Rajkot,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,Saurashtra Cricket Association Stadium,...,7,4,97,False,False,False,False,False,False,1491523200
4,2017,Indore,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,Holkar Cricket Stadium,...,8,5,98,False,False,False,False,False,False,1491609600
5,2017,Bangalore,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,M Chinnaswamy Stadium,...,8,5,98,False,False,False,False,False,False,1491609600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7949,2018,Pune,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,field,normal,0,Chennai Super Kings,Maharashtra Cricket Association Stadium,...,20,6,140,False,False,False,False,False,False,1526774400
7950,2018,Mumbai,Sunrisers Hyderabad,Chennai Super Kings,Chennai Super Kings,field,normal,0,Chennai Super Kings,Wankhede Stadium,...,22,1,142,False,False,False,False,False,False,1526947200
7951,2018,Kolkata,Kolkata Knight Riders,Rajasthan Royals,Rajasthan Royals,field,normal,0,Kolkata Knight Riders,Eden Gardens,...,23,2,143,False,False,False,False,False,False,1527033600


#### Since the secound row in the test set consists of some missing values we are going to drop it and predict the rest two cloumns.

In [12]:
test=test.drop(test.index[1])
test

,season,city,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,venue,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
id,,,,,,,,,,,,,,,,,,,,,
11145,2019,Mohali,Mumbai Indians,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,NaN,...,30,5,89,False,False,False,False,False,False,1553904000
11415,2019,Hyderabad,Mumbai Indians,Chennai Super Kings,Mumbai Indians,bat,normal,0,Mumbai Indians,NaN,...,5,3,339,False,False,False,False,False,False,1575504000


#### 'proc_df' takes a data frame df and splits off the response variable, and changes the df into an entirely numeric dataframe. It also deals with the NaN and NULL vales and fils them with the mediam value,but since we already dealth with the NULL vales this function is going to ignore it.

In [15]:
X_train, y_train, nas = proc_df(data1, 'winner')
X_test,y_test , nas = proc_df(test, 'winner')

After this our data if ready to fit into a Machine learning model.
in our case we choose random forest since it can deal with almost all forms of datasets and gives optimun results, in further improvements we will be using Models like XGboost etc to test them further.  

In [16]:
m = RandomForestClassifier(n_estimators=100,n_jobs=-1)
m.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

### Since the test data was split and saves into X_test we are going to pass X_test onto the predict function. 

In [17]:
y_pred=m.predict(X_test)

### For our test data we only considered 2 matches, one from the middle of the season and the other one is the finals.

In [18]:
y_pred

array([4, 7], dtype=int8)

#### The output we have here is array because 'Proc_df' converted the catagorical variables to numbers. Now our task is to decode the arrays to team names.

#### Lets first look at how the function train_cats has encoded our target variable 'winner'.  

In [21]:
data1.winner.cat.categories

Index(['Chennai Super Kings', 'Deccan Chargers', 'Delhi Daredevils',
       'Gujarat Lions', 'Kings XI Punjab', 'Kochi Tuskers Kerala',
       'Kolkata Knight Riders', 'Mumbai Indians', 'Pune Warriors',
       'Rajasthan Royals', 'Rising Pune Supergiant', 'Rising Pune Supergiants',
       'Royal Challengers Bangalore', 'Sunrisers Hyderabad'],
      dtype='object')

#### Since the y_pred is a numpy array we are converting it into a dataframe.

In [22]:
y_pred=pd.DataFrame(y_pred)
y_pred=y_pred.rename(columns={0: "winner"})

#### Now we have to replace the encodeed arrays to name of the teams.
#### we do that by using the function 'replace' and passing the arrays and their replacements.

In [23]:
y_pred.replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13],['Chennai Super Kings', 'Deccan Chargers', 'Delhi Daredevils',
       'Gujarat Lions', 'Kings XI Punjab', 'Kochi Tuskers Kerala',
       'Kolkata Knight Riders', 'Mumbai Indians', 'Pune Warriors',
       'Rajasthan Royals', 'Rising Pune Supergiant', 'Rising Pune Supergiants',
       'Royal Challengers Bangalore', 'Sunrisers Hyderabad'])

,winner
0,Kings XI Punjab
1,Mumbai Indians


### As we can see our model Predicted the 2019 finals correct.

#### For now our model is a bit fragile, we can make it robust by improving its parameters. 
#### 7 iterations out of 10 are correct.

#### Furthur improvements 
#### Hyper prameter tuning of the model.
#### using other models like XGBoost,ADA boost etc..